# Task C:
Implementing 2-layered Convolutional Model trained to predict LouisaML_Reduced Dataset


In [39]:
import numpy as np
import scipy.io as scio
import h5py
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import *
from keras.utils.np_utils import to_categorical
from keras.callbacks import Callback
from sklearn import metrics
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import random, keras
import time
import _pickle as cPickle

## Data Pre-processing
Load Dataset, inspect keys, load dataset as array of miostruct objects and inspect its shape

In [8]:
S=scio.loadmat('LouisaML_Reduced.mat');
print(S.keys()) 
data=S['dataset'] 
print(data.shape)

dict_keys(['__globals__', '__header__', '__version__', 'dataset'])
(8, 20)


Extract modulation types(label), sigvalue (A) and snr information (snr) from data, i.e. get rid of elements that we do not need and flatten the indexing. Make sure the dataset is not sparse (filled with zeros) or overlaps (double 1s in label).

In [44]:
label=data[:,:]['label']
#label_before_unravel has 3 level indexes
print('label[0][5][514]',label[0][5][514])
print('label.shape',label.shape)
#label.ravel() returns contiguous flattened array
label=label.ravel()
print('label_after_unravel',label[20][999])
lbl=np.vstack(label)
print('lbl',lbl, len(lbl))
print('lbl.shape',lbl.shape)
print('lbl[5514]',lbl[5514])

A=data[:,:]['A']
print('A[7][18].shape', A[7][18].shape)
print(A[7].shape)
A=A.ravel()
A=np.vstack(A)
print('label',lbl)
print(label[7][2].shape)
print("A.shape:",A.shape)

snr=data[:,:]['snr']
snr=snr.ravel()
snr=np.vstack(snr)
print('snr',snr, len(snr))
print('snr.shape and type',snr.shape, type(snr))
print('snr[15984]',snr[15984])
snr=snr.tolist()


label[0][5][514] [1 0 0 0]
label.shape (8, 20)
label_after_unravel [0 1 0 0]
lbl [[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ..., 
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]] 160000
lbl.shape (160000, 4)
lbl[5514] [1 0 0 0]
A[7][18].shape (1000, 2, 128)
(20,)
label [[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ..., 
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]
(4,)
A.shape: (160000, 2, 128)
snr [[ 0]
 [ 0]
 [ 0]
 ..., 
 [19]
 [19]
 [19]] 160000
snr.shape and type (160000, 1) <class 'numpy.ndarray'>
snr[15984] [15]


Generating labels in dictionary format, i.e. keys in the form of tuple (modulation type, SNR type). Total number of keys should be No. of modulation types * No. of SNR types

In [11]:
n=['BPSK','QPSK','16QAM','64QAM']
k=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
keys = [(n,k) for k in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19] for n in ['BPSK','QPSK','16QAM','64QAM']]

labelDictionary={}
for i in range(len(keys)):
    for j in range(1000):
        labelDictionary[keys[i]]=A[j]
print(labelDictionary.keys(), len(labelDictionary.keys()))

dict_keys([('64QAM', 19), ('16QAM', 9), ('QPSK', 2), ('BPSK', 6), ('QPSK', 12), ('QPSK', 3), ('64QAM', 1), ('BPSK', 17), ('16QAM', 4), ('64QAM', 0), ('64QAM', 15), ('BPSK', 3), ('16QAM', 1), ('QPSK', 4), ('QPSK', 18), ('64QAM', 16), ('16QAM', 8), ('64QAM', 8), ('16QAM', 14), ('BPSK', 7), ('16QAM', 5), ('BPSK', 8), ('BPSK', 14), ('64QAM', 13), ('QPSK', 9), ('16QAM', 11), ('16QAM', 18), ('16QAM', 6), ('64QAM', 2), ('QPSK', 13), ('QPSK', 5), ('QPSK', 19), ('64QAM', 17), ('QPSK', 6), ('BPSK', 15), ('BPSK', 0), ('BPSK', 12), ('QPSK', 1), ('64QAM', 10), ('64QAM', 7), ('16QAM', 0), ('BPSK', 19), ('64QAM', 4), ('16QAM', 17), ('BPSK', 4), ('BPSK', 13), ('64QAM', 9), ('QPSK', 8), ('64QAM', 3), ('QPSK', 10), ('64QAM', 6), ('64QAM', 12), ('QPSK', 15), ('QPSK', 14), ('16QAM', 19), ('BPSK', 1), ('16QAM', 3), ('BPSK', 10), ('64QAM', 18), ('16QAM', 10), ('16QAM', 16), ('16QAM', 13), ('BPSK', 5), ('16QAM', 2), ('16QAM', 7), ('QPSK', 0), ('BPSK', 16), ('QPSK', 11), ('64QAM', 11), ('QPSK', 17), ('16QAM',

Data Split for Training, Test, Validation

In [15]:
np.random.seed(2016)
n_examples = A.shape[0]
print("A.shape[0]:", n_examples)
n_train = int(n_examples * 0.6)
train_idx = list(np.random.choice(n_examples, size=n_train, replace=False)) #changed array to list
n_validation = int(n_examples*0.2)
validation_set = list(set(range(0,n_examples))-set(train_idx))
validation_idx = list(np.random.choice(validation_set, size=n_validation, replace=False)) #hold-out values for validation
test_idx = list(set(range(0,n_examples))- set(train_idx) - set(validation_idx))
A_train=np.zeros((96000,2,128),np.float64)
Y_train=np.zeros((96000,4),np.float64)
A_test=np.zeros((32000,2,128),np.float64)
Y_test=np.zeros((32000,4),np.float64)
A_validation=np.zeros((32000,2,128),np.float64)
Y_validation=np.zeros((32000,4),np.float64)

#More Shuffling in place
np.random.shuffle(train_idx)
np.random.shuffle(test_idx)
np.random.shuffle(validation_idx)

#Filling in A and Y tensors
z=0
for p in train_idx:
    A_train[z] = A[p]
    Y_train[z] = lbl[p]
    z=z+1

z=0
for q in test_idx:
    A_test[z] = A[q]
    Y_test[z] = lbl[q]
    z=z+1

z=0
for r in validation_idx:
    A_validation[z] = A[r]
    Y_validation[z] = lbl[r]
    z=z+1

#For verification, A[last_validation] = A_validation[-1] and lbl[last_validation] = Y_validation[-1] 
last_validation=validation_idx[-1]
print("A_validation :", A[last_validation])
print("Y_validation :", lbl[last_validation])
print("A_validation :",A_validation[-1])
print("Y_validation :", Y_validation[-1])

A.shape[0]: 160000
A_validation : [[  4.76589601e-03  -8.39680207e-03  -9.69669676e-04  -1.30235452e-02
   -1.42604281e-02   3.30993651e-03   2.06726595e-03  -4.64013275e-03
    2.73783868e-03  -3.22163011e-03  -4.93991292e-03  -5.30492112e-03
   -3.88759067e-03  -1.35701533e-03  -3.35151521e-03   4.50426370e-03
    1.69548192e-03  -2.01290870e-03  -3.23605546e-03   9.53620540e-05
   -1.02452811e-05   2.22020748e-03  -5.63387386e-04   5.51849686e-03
    1.67515307e-03  -2.02031302e-03  -3.67312348e-03  -5.07519912e-03
    1.34373465e-03  -4.66858843e-03   7.89879036e-03  -2.95486122e-03
   -1.97536626e-03  -3.73153936e-03  -1.76904549e-02  -1.11928855e-02
    5.42259525e-03  -4.79759887e-03  -6.71123337e-03   1.34118527e-02
    1.09320711e-02   2.65144179e-03  -4.69330529e-03  -3.19556194e-03
   -8.00064210e-03  -4.20395124e-03  -7.22356726e-03   9.47140311e-03
    2.06095171e-04  -2.27902868e-04   1.99382043e-03  -5.73761578e-03
    6.01205887e-03  -8.51660023e-04   1.82183350e-03   3

## Training in Keras Model
Prints out Input Shape of Dataset and set up 2-layered CNN Model with Default Adam Optimizer Values, Dropout Rate = 50%, Number of Epochs = 500, Batch Size=1024

In [16]:
in_shp = list(A_train.shape[1:])
print ("A_train.shape, Y_train.shape, in_shp :", A_train.shape,Y_train.shape,in_shp)
classes=n #Classes to be predicted, i.e. the Modulation Types
#Set up some params
adam = adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0)
dr = 0.5 # dropout rate (%)
epochs=500
batch_size= 1024
model = models.Sequential()
model.add(Reshape((1,2,128),input_shape=in_shp))
model.add(ZeroPadding2D((0, 2), data_format='channels_first'))
model.add(Convolution2D(256,(1,3), activation="relu", name="conv1", padding="valid", kernel_initializer='glorot_uniform',data_format='channels_first'))
model.add(Dropout(dr))
model.add(ZeroPadding2D((0, 2), data_format='channels_first'))
model.add(Convolution2D(80,(2, 3), activation="relu", name="conv2", padding="valid", kernel_initializer='glorot_uniform',data_format='channels_first'))
model.add(Dropout(dr))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_normal', name="dense1"))
model.add(Dropout(dr))
model.add(Dense( len(classes), kernel_initializer='he_normal', name="dense2" ))
model.add(Activation('softmax'))
model.add(Reshape([len(classes)]))
#compile/configure models
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"]) 
model.summary()
#t1=time.time()
#lapsed_time=t1-t0
#print("Lapsed time is %0.2f seconds" %lapsed_time)
print("Model set up")
print("Training NN")

A_train.shape, Y_train.shape, in_shp : (96000, 2, 128) (96000, 4) [2, 128]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1, 2, 128)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 1, 2, 132)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 2, 130)       1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256, 2, 130)       0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 256, 2, 134)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 80, 1, 132)        122960    
_________________________________________________________________
d

Check that NN is initialized with random weights and shape of weights are correct

In [17]:
weights = model.get_weights()
w0=weights[0][0][0]
w1=weights[0][0][1]
print("neural net initialised with weights w0: {} and w1:{}".format(w0,w1))
print ("Weight shape", w0.shape, w1.shape)

neural net initialised with weights w0: [[ 0.01964319 -0.02461094  0.07247709 -0.01712509  0.06624751 -0.043476
  -0.00558118  0.0234168  -0.01061016  0.05644912 -0.00027969  0.05206758
   0.03402125  0.01543937  0.00020496  0.03410891 -0.06648343  0.06306066
   0.06809185  0.05163629  0.03959253 -0.02239642  0.0567373   0.05940518
  -0.03394366 -0.0336021   0.04147181 -0.0741792  -0.04119142  0.02938319
  -0.06608526 -0.00865794 -0.0623279  -0.08099447 -0.06418459 -0.01227389
  -0.04108376 -0.06001418  0.01984257 -0.0056485  -0.01141667 -0.02476753
  -0.06403825  0.07930967 -0.06448561 -0.06161448 -0.01666175  0.01328807
   0.02898153  0.06293677  0.06768407  0.07658558  0.06324585 -0.05386682
   0.07946286  0.01518368  0.02786968 -0.00263763  0.07207085 -0.06631574
  -0.00284047  0.02374373  0.05867499  0.01666339 -0.03525848 -0.05773621
   0.02468436  0.05792414  0.02978832  0.01424483  0.07084759 -0.00069081
   0.06827396 -0.02517089  0.04556865  0.00524195 -0.02359899 -0.03497031


Start training and record total training time

In [23]:
t0=time.clock()
filepath='NN1_1024_0.5.weights.h5'
history = model.fit(A_train,Y_train,batch_size=batch_size,epochs=epochs,verbose=2,validation_data=(A_validation, Y_validation),callbacks =[keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])
t1=time.clock()
print("Total Training Time for NN1: ",t1-t0," seconds")

Train on 96000 samples, validate on 32000 samples
Epoch 1/500
20s - loss: 1.3264 - acc: 0.3727 - val_loss: 1.3233 - val_acc: 0.3763
Epoch 2/500
19s - loss: 1.3227 - acc: 0.3734 - val_loss: 1.3310 - val_acc: 0.3763
Epoch 3/500
19s - loss: 1.3202 - acc: 0.3734 - val_loss: 1.3142 - val_acc: 0.3763
Epoch 4/500
19s - loss: 1.3164 - acc: 0.3737 - val_loss: 1.3136 - val_acc: 0.3763
Epoch 5/500
19s - loss: 1.3112 - acc: 0.3752 - val_loss: 1.3039 - val_acc: 0.3782
Epoch 6/500
19s - loss: 1.3048 - acc: 0.3805 - val_loss: 1.2967 - val_acc: 0.3827
Epoch 7/500
20s - loss: 1.2962 - acc: 0.3850 - val_loss: 1.2889 - val_acc: 0.3855
Epoch 8/500
19s - loss: 1.2891 - acc: 0.3894 - val_loss: 1.2810 - val_acc: 0.3943
Epoch 9/500
19s - loss: 1.2786 - acc: 0.3951 - val_loss: 1.2680 - val_acc: 0.4001
Epoch 10/500
19s - loss: 1.2663 - acc: 0.4043 - val_loss: 1.2558 - val_acc: 0.4224
Epoch 11/500
19s - loss: 1.2420 - acc: 0.4244 - val_loss: 1.2226 - val_acc: 0.4361
Epoch 12/500
19s - loss: 1.2053 - acc: 0.4452 

Print scores [Loss, Accuracy] and plot training accuracy and loss curves

In [24]:
score=model.evaluate(A_validation, Y_validation, verbose=0, batch_size=batch_size)
print(score)
#plot accuracy curves
plt.figure(1)
plt.subplot(211)
plt.title('Accuracy plot')
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train','validation'], loc='upper left')

#plot loss curves
plt.subplot(212)
plt.title('Training performance')
plt.plot(history.epoch, history.history['loss'], label='train loss+error')
plt.plot(history.epoch,history.history['val_loss'],label='val_error')
plt.ylabel('Error')
plt.xlabel('Epochs')
plt.legend(['train','validation'], loc='upper left')
plt.show()

[0.73113847827911382, 0.69199999999999995]


## Predicting with Trained Weights
Load weights, start prediction, plot confusion matrix for all SNRs

In [37]:
model.load_weights(filepath)
print("Loaded weights from Disk")

#defines function which return all indices for value
def all_indices(value, qlist):
    indices = []
    idx = -1
    while True:
        try:
            idx = qlist.index(value, idx+1)
            indices.append(idx)
        except ValueError:
            break
    return indices

#defines function to plot confusion matrix
def plot_confusion_matrix(cm, title, cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    #plt.show
    plt.savefig(title,format='png')

#Given A_test, predict Y_hat
test_Y_hat = model.predict(A_test, batch_size=batch_size, verbose=1)
#Initialise zero vectors for confusion matrix ( normalized and before normalization)
conf = np.zeros([len(classes),len(classes)])
confnorm = np.zeros([len(classes),len(classes)])
for m in range(0,A_test.shape[0]):
    n = list(Y_test[m,:]).index(1)
    o = int(np.argmax(test_Y_hat[m,:]))
    conf[n,o] = conf[n,o] + 1
for i in range(0,len(classes)):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

#Plot the first confusion matrix for all SNRs
plot_confusion_matrix(confnorm,title="Confusion_Matrix_for_all_SNRs",labels=classes)

Loaded weights from Disk
31744/32000 [============================>.] - ETA: 0s

Predict and plot Confusion Matrix according to SNRs

In [45]:
#Initialise acc and test_SNRs
acc = {}
test_SNRs=[]

#Generate a list of test_SNRs
for i in test_idx:
    test_SNRs.append(snr[i])

#Extract sigvalues and labels @ SNR
for snr in k:
    test_X_i = A_test[all_indices([snr],test_SNRs)]
    test_Y_i = Y_test[all_indices([snr],test_SNRs)]
    test_Y_i_hat = model.predict(test_X_i)
    conf = np.zeros([len(classes),len(classes)])
    confnorm = np.zeros([len(classes),len(classes)])
    for j in range(0,test_X_i.shape[0]):
        true = list(test_Y_i[j,:]).index([1])
        predicted = int(np.argmax(test_Y_i_hat[j,:]))
        conf[true,predicted] = conf[true,predicted] + 1
    #np.seterr(divide='ignore',invalid='ignore')
    for ct in range(0,len(classes)):
        confnorm[ct,:] = conf[ct,:] / np.sum(conf[ct,:])
    #print("conf",conf)
    #print("confnorm",confnorm)
    plt.figure()
    plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))
    
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    print ("Overall Accuracy: ", cor / (cor+ncor))
    acc[snr] = 1.0*cor/(cor+ncor)
# Save results to a pickle file for plotting later
print (acc)
a=list(map(lambda x:acc[x], k))
print("a",a)
with open('results_cnn2_d0.5.dat','wb') as fd:
    cPickle.dump(("CNN2_080218", 0.5, acc),fd)
# Plot accuracy curve
fig=plt.figure()
plt.plot(k, a)
plt.xlabel("Signal to Noise Ratio")
plt.ylabel("Classification Accuracy")
plt.title("CNN2 Classification Accuracy on Dataset 7.146")
plt.show()
fig.savefig('Accplot.png')


/home/louisa/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Overall Accuracy:  0.694670846395
Overall Accuracy:  0.774412855377
Overall Accuracy:  0.668918918919
Overall Accuracy:  0.683076923077
Overall Accuracy:  0.653775322284
Overall Accuracy:  0.721706864564
Overall Accuracy:  0.57087628866
Overall Accuracy:  0.658491722869
Overall Accuracy:  0.71835443038
Overall Accuracy:  0.656015037594
Overall Accuracy:  0.684345047923
Overall Accuracy:  0.787411689146
Overall Accuracy:  0.686261980831
Overall Accuracy:  0.630692451653
Overall Accuracy:  0.757748260595
Overall Accuracy:  0.688649334179
Overall Accuracy:  0.744112030554
Overall Accuracy:  0.657734470158
Overall Accuracy:  0.69302615193
Overall Accuracy:  0.723225030084
{0: 0.69467084639498433, 1: 0.77441285537700866, 2: 0.66891891891891897, 3: 0.68307692307692303, 4: 0.65377532228360957, 5: 0.72170686456400746, 6: 0.57087628865979378, 7: 0.65849172286940527, 8: 0.71835443037974689, 9: 0.65601503759398494, 10: 0.68434504792332274, 11: 0.78741168914579318, 12: 0.68626198083067091, 13: 0.6